In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchdiffeq import odeint
from sklearn.model_selection import train_test_split

In [4]:
X = np.load("C:\\Users\\ankit\\OneDrive\\Desktop\\Capstone\\trial_05\\X_preprocessed.npy")
y = np.load("C:\\Users\\ankit\\OneDrive\\Desktop\\Capstone\\trial_05\\y_preprocessed.npy")

# Sanitize NaN/Inf in X
if np.any(~np.isfinite(X)):
    print("⚠️ Warning: NaNs/Infs detected in X. Fixing...")
    # forward fill NaNs
    for i in range(X.shape[2]):
        col = X[:, :, i]
        mask = ~np.isfinite(col)
        if mask.any():
            col[mask] = np.nan
            # forward/back fill
            col = np.where(np.isnan(col), np.nanmean(col), col)
        X[:, :, i] = np.clip(col, -10, 10)

# Sanitize NaN/Inf in y
if np.any(~np.isfinite(y)):
    print("⚠️ Warning: NaNs/Infs detected in y. Fixing...")
    y = np.where(np.isfinite(y), y, np.nanmean(y))
    y = np.clip(y, -10, 10)

print("✅ Data loaded and sanitized")

⚠️ Warning: NaNs/Infs detected in X. Fixing...
✅ Data loaded and sanitized


In [5]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).unsqueeze(-1)

In [6]:
class LTCCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.W = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.time_constant = nn.Parameter(torch.ones(hidden_dim))

    def forward(self, t, h):
        # h: (batch, hidden_dim)
        x = self.current_input  # set externally
        combined = torch.cat([x, h], dim=-1)
        dh = torch.tanh(self.W(combined)) - h
        return dh / (torch.abs(self.time_constant) + 1e-6)

In [7]:
class StackedLTC(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, t_span=[0.0, 1.0]):
        super().__init__()
        self.layers = nn.ModuleList()
        self.t_span = t_span
        prev_dim = input_dim
        for hdim in hidden_dims:
            self.layers.append(LTCCell(prev_dim, hdim))
            prev_dim = hdim
        self.head = nn.Linear(prev_dim, output_dim)

    def forward(self, x):
        device = x.device
        batch_size, seq_len, _ = x.size()
        hs = [torch.zeros(batch_size, layer.hidden_dim, device=device) for layer in self.layers]

        for t in range(seq_len):
            cur = x[:, t, :]
            for li, layer in enumerate(self.layers):
                if li == 0:
                    layer.current_input = cur
                else:
                    layer.current_input = hs[li-1]
                # ✅ FIX: ensure t_span is tensor
                t_span = torch.tensor(self.t_span, dtype=torch.float32, device=cur.device)
                h_next = odeint(layer, hs[li], t_span)[-1]
                hs[li] = torch.tanh(h_next)

        final_h = hs[-1]
        out = self.head(final_h)
        return out

In [8]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = StackedLTC(input_dim=X.shape[2], hidden_dims=[64, 64, 32], output_dim=1).to(DEVICE)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_ds = torch.utils.data.TensorDataset(X_train, y_train)
val_ds = torch.utils.data.TensorDataset(X_val, y_val)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=32)

C:\Users\ankit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
import pandas as pd
from torchmetrics import MeanAbsoluteError, MeanSquaredError, R2Score

EPOCHS = 20
final_model_path = "lnn_final_model.pth"

# 📊 Store results for each epoch
history = {
    "Epoch": [],
    "Train Loss": [],
    "Val Loss": [],
    "Train MSE": [],
    "Val MSE": [],
    "Train MAE": [],
    "Val MAE": [],
    "Train R²": [],
    "Val R²": []
}

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0
    
    # Train metrics
    train_mse = MeanSquaredError().to(DEVICE)
    train_mae = MeanAbsoluteError().to(DEVICE)
    train_r2  = R2Score().to(DEVICE)

    for Xb, yb in train_loader:
        Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        out = model(Xb)
        loss = criterion(out, yb)
        if torch.isnan(loss):
            print("❌ NaN loss detected! Breaking...")
            break
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item() * Xb.size(0)
        
        train_mse.update(out, yb)
        train_mae.update(out, yb)
        train_r2.update(out, yb)
    
    train_loss = train_loss / len(train_ds)
    train_mse_val = train_mse.compute().item()
    train_mae_val = train_mae.compute().item()
    train_r2_val  = train_r2.compute().item()

    # -----------------------
    # Validation
    # -----------------------
    model.eval()
    val_loss = 0
    
    val_mse = MeanSquaredError().to(DEVICE)
    val_mae = MeanAbsoluteError().to(DEVICE)
    val_r2  = R2Score().to(DEVICE)
    
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(DEVICE), yb.to(DEVICE)
            out = model(Xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * Xb.size(0)
            
            val_mse.update(out, yb)
            val_mae.update(out, yb)
            val_r2.update(out, yb)
    
    val_loss = val_loss / len(val_ds)
    val_mse_val = val_mse.compute().item()
    val_mae_val = val_mae.compute().item()
    val_r2_val  = val_r2.compute().item()

    # -----------------------
    # Save results
    # -----------------------
    history["Epoch"].append(epoch)
    history["Train Loss"].append(train_loss)
    history["Val Loss"].append(val_loss)
    history["Train MSE"].append(train_mse_val)
    history["Val MSE"].append(val_mse_val)
    history["Train MAE"].append(train_mae_val)
    history["Val MAE"].append(val_mae_val)
    history["Train R²"].append(train_r2_val)
    history["Val R²"].append(val_r2_val)

    print(f"Epoch {epoch}/{EPOCHS} | "
          f"Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | "
          f"Train MSE: {train_mse_val:.6f} | Val MSE: {val_mse_val:.6f} | "
          f"Train MAE: {train_mae_val:.6f} | Val MAE: {val_mae_val:.6f} | "
          f"Train R²: {train_r2_val:.6f} | Val R²: {val_r2_val:.6f}")

# -----------------------
# Save model AFTER training
# -----------------------
torch.save(model.state_dict(), final_model_path)
print(f"✅ Training finished. Final model saved to {final_model_path}")

# -----------------------
# 📊 Tabulate results
# -----------------------
results_df = pd.DataFrame(history)
print("\n📊 Final Training Results:")
print(results_df)

# Optionally save to CSV/Excel
results_df.to_csv("training_results.csv", index=False)


Epoch 1/20 | Train Loss: 0.002184 | Val Loss: 0.005731 | Train MSE: 0.002184 | Val MSE: 0.005731 | Train MAE: 0.035644 | Val MAE: 0.057007 | Train R²: 0.938654 | Val R²: 0.751910
Epoch 2/20 | Train Loss: 0.001307 | Val Loss: 0.003115 | Train MSE: 0.001307 | Val MSE: 0.003115 | Train MAE: 0.026781 | Val MAE: 0.043140 | Train R²: 0.963271 | Val R²: 0.865147
Epoch 3/20 | Train Loss: 0.001013 | Val Loss: 0.002334 | Train MSE: 0.001013 | Val MSE: 0.002334 | Train MAE: 0.023619 | Val MAE: 0.036686 | Train R²: 0.971533 | Val R²: 0.898934
Epoch 4/20 | Train Loss: 0.001028 | Val Loss: 0.002289 | Train MSE: 0.001028 | Val MSE: 0.002289 | Train MAE: 0.024937 | Val MAE: 0.036060 | Train R²: 0.971109 | Val R²: 0.900895
Epoch 5/20 | Train Loss: 0.000899 | Val Loss: 0.001312 | Train MSE: 0.000899 | Val MSE: 0.001312 | Train MAE: 0.022392 | Val MAE: 0.029300 | Train R²: 0.974743 | Val R²: 0.943203
Epoch 6/20 | Train Loss: 0.000924 | Val Loss: 0.002215 | Train MSE: 0.000924 | Val MSE: 0.002215 | Train 

In [9]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = StackedLTC(input_dim=X.shape[2], hidden_dims=[64, 64, 32], output_dim=1).to(DEVICE)
model.load_state_dict(torch.load("C:\\Users\\ankit\\OneDrive\\Desktop\\Capstone\\trial_05\\lnn_final_model.pth", map_location=DEVICE))
model.eval()

StackedLTC(
  (layers): ModuleList(
    (0): LTCCell(
      (W): Linear(in_features=75, out_features=64, bias=True)
    )
    (1): LTCCell(
      (W): Linear(in_features=128, out_features=64, bias=True)
    )
    (2): LTCCell(
      (W): Linear(in_features=96, out_features=32, bias=True)
    )
  )
  (head): Linear(in_features=32, out_features=1, bias=True)
)

In [12]:
import pandas as pd


In [13]:
df = pd.read_csv("C:\\Users\\ankit\\OneDrive\\Desktop\\Capstone\\trial_05\\TCS_2020_present.csv")  
# Assume df has columns: Date, Open, High, Low, Close, Volume

# Feature preparation (adjust same way you preprocessed X_preprocessed.npy)
# For demo, we use last N candles as input sequence
SEQ_LEN = 30
features = ["Open", "High", "Low", "Close", "Volume"]
X, y = [], []